In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout,Dense,ReLU,Softmax

In [2]:
df = pd.read_csv('D:/Final Year Project/Express-U/data/preprocessed/canny_labels.csv',index_col=False)
df

,file_paths,labels
0,D:/Final Year Project/Express-U/data/preproces...,1
1,D:/Final Year Project/Express-U/data/preproces...,1
2,D:/Final Year Project/Express-U/data/preproces...,1
3,D:/Final Year Project/Express-U/data/preproces...,1
4,D:/Final Year Project/Express-U/data/preproces...,1
...,...,...
41995,D:/Final Year Project/Express-U/data/preproces...,Z
41996,D:/Final Year Project/Express-U/data/preproces...,Z
41997,D:/Final Year Project/Express-U/data/preproces...,Z
41998,D:/Final Year Project/Express-U/data/preproces...,Z


In [3]:
X_Train,X_Test = train_test_split(df,train_size=0.8,random_state=42,shuffle=True)

In [4]:
print(X_Train.shape)
print(X_Test.shape)

(33600, 2)
(8400, 2)


In [5]:
Train_image_generator = ImageDataGenerator(rescale=1/255,
                                        zoom_range=0.5,
                                        brightness_range=[0.6,1.0],
                                        rotation_range=20,
                                        validation_split=0.1)
Validation_image_generator = ImageDataGenerator(rescale=1/255)

In [6]:
Train_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   subset="training")

Validation_Set = Validation_image_generator.flow_from_dataframe(dataframe=X_Test,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   shuffle=False)

Found 30240 validated image filenames belonging to 35 classes.
Found 8400 validated image filenames belonging to 35 classes.


In [7]:
inputshape = (256,256,1)
outputshape = 35

model = Sequential()
model.add(Conv2D(input_shape=inputshape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(outputshape,activation="softmax"))


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                    

In [9]:
lr=0.001
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

In [ ]:
history = model.fit(Train_Set,epochs=10,batch_size=40,verbose=1,validation_data=Validation_Set)

Epoch 1/10
